In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA

# Data inladen
train_data = pd.read_csv('train_data.csv')

# Data voorbereiden
training_label = train_data['label']
training_values = train_data.drop(columns=['label'], axis=1)

# Gebruik Random Forest om feature importances te verkrijgen
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(training_values, training_label)

# Verkrijg de belangrijkheid van de features
feature_importances = rf.feature_importances_

# Selecteer de top K belangrijkste features (bijv. 10 belangrijkste)
k = 10  # Pas dit aan afhankelijk van hoeveel features je wilt behouden
indices = np.argsort(feature_importances)[::-1][:k]
training_values_selected = training_values.iloc[:, indices]

# Als er nog steeds meer dan 2 features zijn, reduceren we naar 2D met PCA
if training_values_selected.shape[1] > 2:
    pca = PCA(n_components=2)
    training_values_selected = pca.fit_transform(training_values_selected)

# Split de data in training en test sets
X_train, X_test, y_train, y_test = train_test_split(training_values_selected, training_label, test_size=0.3, random_state=42)

# Maak een Soft Margin SVM model aan
svm = SVC(C=1.0, kernel='linear', random_state=42)

# Train het model
svm.fit(X_train, y_train)

# Voorspel op de test set
y_pred = svm.predict(X_test)

# Bereken de nauwkeurigheid van het model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy van de Soft Margin SVM classifier met feature selectie: {accuracy:.2f}')

# Plot de beslissinggrens en de data
plt.figure(figsize=(10, 6))

# Bereken de besluitlijn (marge) voor de classifier
x_min, x_max = X_train[:, 0].min() - 1, X_train[:, 0].max() + 1
y_min, y_max = X_train[:, 1].min() - 1, X_train[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.1),
                     np.arange(y_min, y_max, 0.1))

Z = svm.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

# Plot de beslissinggrens
plt.contourf(xx, yy, Z, alpha=0.8, cmap=plt.cm.coolwarm)

# Plot de trainingsdata
plt.scatter(X_train[:, 0], X_train[:, 1], c=y_train, marker='o', edgecolors='k', cmap=plt.cm.coolwarm, s=50)

# Voeg titels en labels toe
plt.title('Besluitlijn van de Soft Margin SVM met Feature Selectie via Random Forest', fontsize=14)
plt.xlabel('Feature 1', fontsize=12)
plt.ylabel('Feature 2', fontsize=12)
plt.colorbar()

plt.show()
